# Project Introduction: Integrating ADX and Parabolic SAR for Stock Analysis

## Objective
The objective of this project is to develop a reliable stock screening tool that identifies potential buy and sell signals using two powerful technical indicators: the Average Directional Index (ADX) and the Parabolic Stop and Reverse (SAR). By combining these indicators, we aim to enhance the accuracy of our signals, leveraging the strengths of both trend strength and trend-following analysis.

## Background
Technical analysis involves using historical price and volume data to forecast future price movements. Among the myriad of technical indicators available, ADX and SAR are two widely respected tools that traders use to assess market conditions.

## Why ADX and Parabolic SAR?

### Average Directional Index (ADX)
- **Purpose:** The ADX is used to quantify the strength of a trend, regardless of its direction. It helps traders understand whether a market is trending or ranging.
- **Components:**
  - **ADX Line:** Indicates the strength of a trend. A value above 25 typically signifies a strong trend, while a value below 20 indicates a weak or non-trending market.
  - **Plus Directional Indicator (+DI):** Measures the presence of upward trend movement.
  - **Minus Directional Indicator (-DI):** Measures the presence of downward trend movement.
- **Signals:**
  - **Strong Trend:** ADX above 25.
  - **Weak Trend:** ADX below 20.

### Parabolic Stop and Reverse (SAR)
- **Purpose:** The Parabolic SAR is a trend-following indicator that provides potential entry and exit points based on price reversals.
- **Components:**
  - **SAR Line:** A series of dots placed either above or below the price chart, indicating potential reversal points.
- **Signals:**
  - **Buy Signal:** When the price crosses above the SAR line.
  - **Sell Signal:** When the price crosses below the SAR line.

## Combining ADX and Parabolic SAR
Using these indicators together can help confirm signals and reduce false positives:

- **Enhanced Accuracy:** While the ADX helps identify the strength of a trend, SAR provides precise entry and exit points during these trends.
- **Filter for False Signals:** The SAR can sometimes give premature signals in weak trends. By requiring confirmation through ADX's trend strength, we filter out some of these false signals.

## Methodology
- **Data Collection:** We collect historical price data for a selection of stocks over a specified period.
- **Indicator Calculation:** We calculate the ADX and SAR for each stock.
- **Stock Screening:** We screen stocks based on the following criteria:
  - **Buy Criteria:** ADX above 30, indicating a strong trend, and the price crosses above the SAR line.
  - **Sell Criteria:** ADX above 30, indicating a strong trend, and the price crosses below the SAR line.
- **Visualization and Export:** We visualize the results and export the screened stocks to a CSV file for further analysis.

## Data Source and Flexibility
In this analysis, we will be using data from the FTSE 250 companies. However, feel free to use any dataset you prefer by simply changing the CSV file name in the script. Ensure that your dataset contains a column named "Ticker". 

## Conclusion
This project aims to leverage the complementary strengths of ADX and Parabolic SAR to create a thought-provoking stock screening tool. By considering both trend strength and precise entry/exit points, we can provide more potential buy and sell signals. The chosen thresholds and periods can be customized to suit your preferences and further investigations, enabling a flexible approach to stock analysis.


In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import contextlib
import sys
import os
from tqdm import tqdm

# Function to suppress print statements
@contextlib.contextmanager
def suppress_output():
    with open(os.devnull, 'w') as devnull:
        old_stdout = sys.stdout
        old_stderr = sys.stderr
        sys.stdout = devnull
        sys.stderr = devnull
        try:
            yield
        finally:
            sys.stdout = old_stdout
            sys.stderr = old_stderr

# Function to calculate ADX
def calculate_adx(data, window=14):
    high = data['High']
    low = data['Low']
    close = data['Close']
    
    plus_dm = high.diff()
    minus_dm = low.diff()
    
    plus_dm[plus_dm < 0] = 0
    minus_dm[minus_dm > 0] = 0
    
    tr1 = pd.DataFrame(high - low)
    tr2 = pd.DataFrame(abs(high - close.shift(1)))
    tr3 = pd.DataFrame(abs(low - close.shift(1)))
    tr = pd.concat([tr1, tr2, tr3], axis=1, join='inner').max(axis=1)
    
    atr = tr.rolling(window).mean()
    
    plus_di = 100 * (plus_dm.ewm(alpha=1/window).mean() / atr)
    minus_di = abs(100 * (minus_dm.ewm(alpha=1/window).mean() / atr))
    
    dx = (abs(plus_di - minus_di) / abs(plus_di + minus_di)) * 100
    adx = ((dx.shift(1) * (window - 1)) + dx) / window
    data['ADX'] = adx.ewm(alpha=1/window).mean()
    return data

# Function to calculate SAR
def calculate_sar(data, acceleration=0.02, maximum=0.2):
    high = data['High']
    low = data['Low']
    close = data['Close']
    
    sar = close.copy()
    long = True
    af = acceleration
    ep = low.iloc[0]
    sar.iloc[0] = ep
    
    for i in range(1, len(sar)):
        if long:
            ep = max(high.iloc[i], ep)
        else:
            ep = min(low.iloc[i], ep)
        
        sar.iloc[i] = sar.iloc[i-1] + af * (ep - sar.iloc[i-1])
        
        reverse = False
        if long:
            if low.iloc[i] < sar.iloc[i]:
                long = False
                reverse = True
                sar.iloc[i] = ep
                ep = low.iloc[i]
                af = acceleration
        else:
            if high.iloc[i] > sar.iloc[i]:
                long = True
                reverse = True
                sar.iloc[i] = ep
                ep = high.iloc[i]
                af = acceleration
        
        if not reverse:
            if long:
                if high.iloc[i] > ep:
                    ep = high.iloc[i]
                    af = min(af + acceleration, maximum)
            else:
                if low.iloc[i] < ep:
                    ep = low.iloc[i]
                    af = min(af + acceleration, maximum)
    
    data['SAR'] = sar
    return data

# Function to analyze stock for ADX and SAR
def analyze_stock_adx_sar(symbol):
    try:
        
        end_date = datetime.now() - timedelta(days=1)
        start_date = end_date - timedelta(days=9*30)  

        # Suppress yfinance output
        with suppress_output():
            data = yf.download(symbol, start=start_date.strftime('%Y-%m-%d'), end=end_date.strftime('%Y-%m-%d'))
        
        if data.empty:
            print(f"No data fetched for {symbol}")
            return None

        # Calculate ADX and SAR
        data = calculate_adx(data)
        data = calculate_sar(data)

        data.dropna(subset=['ADX', 'SAR'], inplace=True)

        if data.empty:
            print(f"Insufficient data after calculating indicators for {symbol}")
            return None

        # Determine if the stock meets any of the criteria
        latest_adx = data['ADX'].iloc[-1]
        latest_close = data['Close'].iloc[-1]
        latest_sar = data['SAR'].iloc[-1]

        # Criteria: ADX > 25 indicates a strong trend
        adx_strong_trend = latest_adx > 30

        # Criteria: Close price crossing SAR
        buy_signal = adx_strong_trend and latest_close > latest_sar
        sell_signal = adx_strong_trend and latest_close < latest_sar

        # Determine if the stock meets any of the criteria
        meets_criteria = buy_signal or sell_signal

        return {
            "Symbol": symbol,
            "Data": data,
            "ADX": latest_adx,
            "Close": latest_close,
            "SAR": latest_sar,
            "ADX_Strong_Trend": adx_strong_trend,
            "Buy_Signal": buy_signal,
            "Sell_Signal": sell_signal,
            "Meets Criteria": meets_criteria
        }

    except Exception as e:
        print(f"Error analyzing stock {symbol}: {e}")
        return None

# Load tickers from the CSV file
tickers_df = pd.read_csv('ftse_250_tickers.csv')
symbols = tickers_df['Ticker'].tolist()

# Initialize list to store results

results = []

# Analyze each stock
#for symbol in symbols:
for symbol in tqdm(symbols, desc="Processing stocks"):
    stock_data = analyze_stock_adx_sar(symbol)
    if stock_data is not None and stock_data["Meets Criteria"]:  # Only include stocks meeting the criteria
        results.append(stock_data)

# Convert results to DataFrame
results_df = pd.DataFrame([{
    "Symbol": r["Symbol"],
    "ADX": r["ADX"],
    "Close": r["Close"],
    "SAR": r["SAR"],
    "ADX_Strong_Trend": r["ADX_Strong_Trend"],
    "Buy_Signal": r["Buy_Signal"],
    "Sell_Signal": r["Sell_Signal"],
    "Meets Criteria": r["Meets Criteria"]
} for r in results])

# Export results to CSV
results_df.to_csv("adx_sar_screener_results.csv", index=False)

results_df.index = np.arange(1, len(results_df) + 1)

# Print results
results_df



In [ ]:
# Calculate and print the detailed results
total_tickers = len(tickers_df)
tickers_meeting_criteria = len(results_df)
percentage_meeting_criteria = (tickers_meeting_criteria / total_tickers) * 100

# Print results
print("My preference is a 5-20% threshold which strikes a balance, ensuring a\nfocused yet diverse selection of stocks to allow further investigations.")
print(f"Tickers meeting criteria: {tickers_meeting_criteria}")
print(f"Percentage of shares that pass the criteria: {percentage_meeting_criteria:.2f}%")

# Determine if the percentage is within the desired filter range
if 5 <= percentage_meeting_criteria <= 20:
    print("The percentage of shares meeting the criteria is within the desirable range (5-20%).")
else:
    print("The percentage of shares meeting the criteria is outside the desirable range (5-15%).")




In [ ]:
# Function to plot ADX and SAR indicators

# Function to print basic financial data
def print_financial_data(ticker):
    stock = yf.Ticker(ticker)
    info = stock.info

    def format_value(value, currency=False):
        if isinstance(value, (int, float)):
            formatted_value = f"{value:,}"
            if currency:
                formatted_value = f"£{formatted_value}"
            return formatted_value
        return value

    market_cap = format_value(info.get('marketCap', 'N/A'), currency=True)
    shares_outstanding = format_value(info.get('sharesOutstanding', 'N/A'))
    year_high = info.get('fiftyTwoWeekHigh', 'N/A')
    year_low = info.get('fiftyTwoWeekLow', 'N/A')
    pe_ratio = info.get('trailingPE', 'N/A')
    eps = info.get('trailingEps', 'N/A')
    dividend_yield = info.get('dividendYield', 'N/A')
    payout_ratio = info.get('payoutRatio', 'N/A')
    revenue = format_value(info.get('totalRevenue', 'N/A'), currency=True)
    gross_profit = format_value(info.get('grossProfits', 'N/A'), currency=True)
    net_income = format_value(info.get('netIncomeToCommon', 'N/A'), currency=True)
    total_debt = format_value(info.get('totalDebt', 'N/A'), currency=True)
    operating_cash_flow = format_value(info.get('operatingCashflow', 'N/A'), currency=True)
    free_cash_flow = format_value(info.get('freeCashflow', 'N/A'), currency=True)
    sector = info.get('sector', 'N/A')
    industry = info.get('industry', 'N/A')
    description = info.get('longBusinessSummary', 'N/A')

    print(f"{'Ticker:':<25} {ticker}")
    print(f"{'Company:':<25} {info.get('shortName', 'N/A')}")
    print(f"{'Sector:':<25} {sector}")
    print(f"{'Industry:':<25} {industry}")
    print(f"{'Market Cap:':<25} {market_cap}")
    print(f"{'Shares Outstanding:':<25} {shares_outstanding}")
    print(f"{'52-Week High:':<25} {year_high}")
    print(f"{'52-Week Low:':<25} {year_low}")
    print(f"{'PE Ratio (TTM):':<25} {pe_ratio}")
    print(f"{'EPS (TTM):':<25} {eps}")
    print(f"{'Dividend Yield:':<25} {dividend_yield}")
    print(f"{'Payout Ratio:':<25} {payout_ratio}")
    print(f"{'Revenue (TTM):':<25} {revenue}")
    print(f"{'Gross Profit (TTM):':<25} {gross_profit}")
    print(f"{'Net Income (TTM):':<25} {net_income}")
    print(f"{'Total Debt:':<25} {total_debt}")
    print(f"{'Operating Cash Flow (TTM):':<25} {operating_cash_flow}")
    print(f"{'Free Cash Flow (TTM):':<25} {free_cash_flow}")
    print(f"\n{'Description:':<25} {description}")  # Print the full description
    print("-" * 50)



def plot_adx_sar(data, symbol):
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))

    # Plot Close Price and SAR
    ax1.plot(data.index, data['Close'], label='Close Price', color='blue')
    ax1.plot(data.index, data['SAR'], label='Parabolic SAR', color='red', linestyle='dotted')
    ax1.set_title(f'{symbol} - Close Price and Parabolic SAR')
    ax1.set_ylabel('Close Price')
    ax1.legend(loc='upper left')
    ax1.grid(True)

    # Plot ADX
    ax2.plot(data.index, data['ADX'], label='ADX', color='purple')
    ax2.axhline(25, color='red', linestyle='--')
    ax2.set_title('ADX')
    ax2.set_ylabel('ADX Value')
    ax2.legend(loc='upper left')
    ax2.grid(True)

    plt.tight_layout()
    plt.show()

# Print the introductory text
print("The following stocks have met the specified criteria. A plot is displayed showing the stock price and its relationship to the associated indicators. Additionally, detailed financial and company information is provided for each stock.\n")



# Iterate through all tickers to plot graphs and print financial data
for stock_data in results:
    symbol = stock_data["Symbol"]
    data = stock_data["Data"]
    plot_adx_sar(data, symbol)
    print_financial_data(symbol)
    
